In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
pd.set_option('display.max_colwidth', 500)
import numpy as np
import time
import requests
import random
import aiohttp
import seaborn as sns
import re
import csv

In [2]:
def formatTrackData(df, index):
    # df.drop(['Average Points', 'Fastest Time', 'Slowest Time'], axis=1, inplace=True)
    melted = df.pivot(index = 'Engine', columns='Track')
    melted = melted.drop('Mirror', axis=0, errors='ignore')
    melted = melted.drop('100cc', axis=0, errors='ignore')
    melted = melted.rename(index={'150cc': index})
    return melted

In [3]:
prefixes = ['GCN', 'DS', 'GBA', 'N64','SNES']

In [4]:
tracks = '''+ Track Name                   Avg Pts    Avg Place    Best Time    # Plays
---------------------------  ---------  -----------  -----------  ---------
1.  GCN Waluigi Stadium          11.25         2.75         2:11          4
2.  N64 DK's Jungle Parkway      10.20         3.40         2:29          5
3.  Mario Circuit                10.00         3.00         1:38          1
4.  N64 Mario Raceway            10.00         3.00         1:53          1
5.  Luigi Circuit                 9.67         4.33         1:19          3
6.  Toad's Factory                9.50         3.75         2:04          4
7.  Maple Treeway                 9.50         4.00         2:37          2
8.  Mushroom Gorge                9.33         4.33         1:56          3
9.  Grumble Volcano               9.17         4.50         2:05          6
10. SNES Ghost Valley 2           9.00         3.67         1:03          3
11. SNES Mario Circuit 3          8.75         5.00         1:32          4
12. Rainbow Road                  8.00         4.00         2:55          1
13. GCN Peach Beach               8.00         5.00         1:28          2
14. N64 Bowser's Castle           7.67         5.17         2:59          6
15. GBA Shy Guy Beach             7.50         5.50         1:43          2
16. Koopa Cape                7.00         5.00         2:47          1
17. DS Desert Hills           7.00         5.00         1:58          1
18. GBA Bowser Castle 3       6.40         5.60         2:39          5
19. DS Delfino Square         5.86         6.71         2:26          7
20. GCN DK Mountain           5.80         6.20         2:31          5
21. DS Peach Gardens          5.00         7.33         2:22          3
22. Bowser's Castle           4.00         7.00         2:46          1
23. Moonview Highway          3.00         8.50         2:08          2
24. DK Summit                 2.33         9.33         2:17          3
25. Dry Dry Ruins             2.25         9.75         2:15          4
26. N64 Sherbet Land          1.00        11.00         2:35          1
27. Moo Moo Meadows           0.00        12.00         1:38          1'''

In [5]:
print(tracks)

+ Track Name                   Avg Pts    Avg Place    Best Time    # Plays
---------------------------  ---------  -----------  -----------  ---------
1.  GCN Waluigi Stadium          11.25         2.75         2:11          4
2.  N64 DK's Jungle Parkway      10.20         3.40         2:29          5
3.  Mario Circuit                10.00         3.00         1:38          1
4.  N64 Mario Raceway            10.00         3.00         1:53          1
5.  Luigi Circuit                 9.67         4.33         1:19          3
6.  Toad's Factory                9.50         3.75         2:04          4
7.  Maple Treeway                 9.50         4.00         2:37          2
8.  Mushroom Gorge                9.33         4.33         1:56          3
9.  Grumble Volcano               9.17         4.50         2:05          6
10. SNES Ghost Valley 2           9.00         3.67         1:03          3
11. SNES Mario Circuit 3          8.75         5.00         1:32          4
12. Rainbow 

## fucntion to parse inputs

In [30]:
def parseInput(input):
    lines = input.split('\n')
    lines = lines[2:]
    tracklist = []
    for line in lines:
        sp = line.split(' ')
        sp = list(filter(bool, sp))
        sp = sp[1:]
        if sp[0] not in prefixes:
            sp.insert(0, 'Wii')
    
        tracklist.append(sp)
    for track in tracklist:
        track[0 : -4] = [" ".join(track[0: -4])]
    # print(track)
    return tracklist

    

# funtion to format data

In [44]:
def formatData(data,allTracks):
    trackData = pd.DataFrame(data, columns=['Track', 'Avg Pts', 'AP', 'BT', 'R'])
    trackData = trackData.drop('Avg Pts', axis=1)
    trackData['Engine'] = '150cc'
    missing = [x for x in allTracks if x not in trackData['Track'].tolist()]
    for t in missing:
        trackData.loc[len(trackData.index)] = [t, '0', '3:00', '0', '150cc']
    return trackData
    
    

## fucn to pivot data

In [35]:
def pivotData(df):
    melted = df.pivot(index = 'Engine', columns='Track')
    melted.columns = [' '.join(col).strip() for col in melted.columns.values]
    return melted
    


In [54]:
def getTime(time):
    min, sec = time.split(':')
    t = (int(min) * 60) + float(sec) 
    
    return t

In [56]:
def typeInput(combodata):
    APcols = combodata.columns[0:32]
    ATcols = combodata.columns[32:64]
    Rcols = combodata.columns[64:96]
    for r in Rcols:
        # combodata[r] = combodata[r].replace(np.nan, 0)
        combodata[r] = combodata[r].astype(int)
    for p in APcols:
        # combodata[p] = combodata[p].replace(np.nan, '13th')
        # combodata[p] = combodata[p].apply(lambda x: re.sub("[^0-9]", "", x)) 
        combodata[p] = combodata[p].astype(int)
    for T in ATcols:
        combodata[T] = combodata[T].apply(lambda x: getTime(x) if x != np.nan else x)
        combodata[T] = combodata[T].astype(float)
    return combodata

In [32]:
trackData = parseInput(tracks)
print(trackData)

[['GCN Waluigi Stadium', '11.25', '2.75', '2:11', '4'], ["N64 DK's Jungle Parkway", '10.20', '3.40', '2:29', '5'], ['Wii Mario Circuit', '10.00', '3.00', '1:38', '1'], ['N64 Mario Raceway', '10.00', '3.00', '1:53', '1'], ['Wii Luigi Circuit', '9.67', '4.33', '1:19', '3'], ["Wii Toad's Factory", '9.50', '3.75', '2:04', '4'], ['Wii Maple Treeway', '9.50', '4.00', '2:37', '2'], ['Wii Mushroom Gorge', '9.33', '4.33', '1:56', '3'], ['Wii Grumble Volcano', '9.17', '4.50', '2:05', '6'], ['SNES Ghost Valley 2', '9.00', '3.67', '1:03', '3'], ['SNES Mario Circuit 3', '8.75', '5.00', '1:32', '4'], ['Wii Rainbow Road', '8.00', '4.00', '2:55', '1'], ['GCN Peach Beach', '8.00', '5.00', '1:28', '2'], ["N64 Bowser's Castle", '7.67', '5.17', '2:59', '6'], ['GBA Shy Guy Beach', '7.50', '5.50', '1:43', '2'], ['Wii Koopa Cape', '7.00', '5.00', '2:47', '1'], ['DS Desert Hills', '7.00', '5.00', '1:58', '1'], ['GBA Bowser Castle 3', '6.40', '5.60', '2:39', '5'], ['DS Delfino Square', '5.86', '6.71', '2:26', 

In [6]:
lines = tracks.split('\n')
lines = lines[2:]
print(len(lines))


27


In [7]:
t1 = lines[2].split(' ')
t1 = list(filter(bool, t1))
t1 = t1[1:]
print(t1)

['Mario', 'Circuit', '10.00', '3.00', '1:38', '1']


In [8]:
tracklist = []
for line in lines:
    sp = line.split(' ')
    sp = list(filter(bool, sp))
    sp = sp[1:]
    if sp[0] in prefixes:
        print('track is a retro')
    else:
        sp.insert(0, 'Wii')
        print(sp[0])
    tracklist.append(sp)
    


track is a retro
track is a retro
Wii
track is a retro
Wii
Wii
Wii
Wii
Wii
track is a retro
track is a retro
Wii
track is a retro
track is a retro
track is a retro
Wii
track is a retro
track is a retro
track is a retro
track is a retro
track is a retro
Wii
Wii
Wii
Wii
track is a retro
Wii


In [9]:
print(tracklist)

[['GCN', 'Waluigi', 'Stadium', '11.25', '2.75', '2:11', '4'], ['N64', "DK's", 'Jungle', 'Parkway', '10.20', '3.40', '2:29', '5'], ['Wii', 'Mario', 'Circuit', '10.00', '3.00', '1:38', '1'], ['N64', 'Mario', 'Raceway', '10.00', '3.00', '1:53', '1'], ['Wii', 'Luigi', 'Circuit', '9.67', '4.33', '1:19', '3'], ['Wii', "Toad's", 'Factory', '9.50', '3.75', '2:04', '4'], ['Wii', 'Maple', 'Treeway', '9.50', '4.00', '2:37', '2'], ['Wii', 'Mushroom', 'Gorge', '9.33', '4.33', '1:56', '3'], ['Wii', 'Grumble', 'Volcano', '9.17', '4.50', '2:05', '6'], ['SNES', 'Ghost', 'Valley', '2', '9.00', '3.67', '1:03', '3'], ['SNES', 'Mario', 'Circuit', '3', '8.75', '5.00', '1:32', '4'], ['Wii', 'Rainbow', 'Road', '8.00', '4.00', '2:55', '1'], ['GCN', 'Peach', 'Beach', '8.00', '5.00', '1:28', '2'], ['N64', "Bowser's", 'Castle', '7.67', '5.17', '2:59', '6'], ['GBA', 'Shy', 'Guy', 'Beach', '7.50', '5.50', '1:43', '2'], ['Wii', 'Koopa', 'Cape', '7.00', '5.00', '2:47', '1'], ['DS', 'Desert', 'Hills', '7.00', '5.00', 

In [10]:
track = tracklist[0]
track[0 : -4] = [" ".join(track[0: -4])]
print(track)

['GCN Waluigi Stadium', '11.25', '2.75', '2:11', '4']


In [11]:
for track in tracklist:
    track[0 : -4] = [" ".join(track[0: -4])]
    print(track)

['GCN Waluigi Stadium', '11.25', '2.75', '2:11', '4']
["N64 DK's Jungle Parkway", '10.20', '3.40', '2:29', '5']
['Wii Mario Circuit', '10.00', '3.00', '1:38', '1']
['N64 Mario Raceway', '10.00', '3.00', '1:53', '1']
['Wii Luigi Circuit', '9.67', '4.33', '1:19', '3']
["Wii Toad's Factory", '9.50', '3.75', '2:04', '4']
['Wii Maple Treeway', '9.50', '4.00', '2:37', '2']
['Wii Mushroom Gorge', '9.33', '4.33', '1:56', '3']
['Wii Grumble Volcano', '9.17', '4.50', '2:05', '6']
['SNES Ghost Valley 2', '9.00', '3.67', '1:03', '3']
['SNES Mario Circuit 3', '8.75', '5.00', '1:32', '4']
['Wii Rainbow Road', '8.00', '4.00', '2:55', '1']
['GCN Peach Beach', '8.00', '5.00', '1:28', '2']
["N64 Bowser's Castle", '7.67', '5.17', '2:59', '6']
['GBA Shy Guy Beach', '7.50', '5.50', '1:43', '2']
['Wii Koopa Cape', '7.00', '5.00', '2:47', '1']
['DS Desert Hills', '7.00', '5.00', '1:58', '1']
['GBA Bowser Castle 3', '6.40', '5.60', '2:39', '5']
['DS Delfino Square', '5.86', '6.71', '2:26', '7']
['GCN DK Mount

In [12]:
testing = pd.DataFrame(tracklist, columns=['Track', 'Avg Pts', 'AP', 'BT', 'R'])
testing = testing.drop('Avg Pts', axis=1)
testing['Engine'] = '150cc'
testing.head()



,Track,AP,BT,R,Engine
0,GCN Waluigi Stadium,2.75,2:11,4,150cc
1,N64 DK's Jungle Parkway,3.40,2:29,5,150cc
2,Wii Mario Circuit,3.00,1:38,1,150cc
3,N64 Mario Raceway,3.00,1:53,1,150cc
4,Wii Luigi Circuit,4.33,1:19,3,150cc


In [13]:
t2 = formatTrackData(testing, 0)
t2.head()

AP                                                       \
Track  DS Delfino Square DS Desert Hills DS Peach Gardens GBA Bowser Castle 3   
Engine                                                                          
0                   6.71            5.00             7.33                5.60   

                                                                              \
Track  GBA Shy Guy Beach GCN DK Mountain GCN Peach Beach GCN Waluigi Stadium   
Engine                                                                         
0                   5.50            6.20            5.00                2.75   

                                                    ...                   R  \
Track  N64 Bowser's Castle N64 DK's Jungle Parkway  ... Wii Grumble Volcano   
Engine                                              ...                       
0                     5.17                    3.40  ...                   6   

                                                                             \
Track  Wii Koopa Cape Wii Luigi Circuit Wii Maple Treeway Wii Mario Circuit   
Engine                                                                        
0                   1                 3                 2                 1   

                                                                    \
Track  Wii Moo Moo Meadows Wii Moonview Highway Wii Mushroom Gorge   
Engine                                                               
0                        1                    2                  3   

                                            
Track  Wii Rainbow Road Wii Toad's Factory  
Engine                                      
0                     1                  4  

[1 rows x 81 columns]

In [14]:
with open('datasets/tracks', 'r') as f:
    reader = csv.reader(f)
    track_names = list(reader)
track_names = track_names[0]
track_names[0]

'DS Delfino Square'

In [15]:
tracks_inc = testing['Track'].to_list()
print(tracks_inc)

['GCN Waluigi Stadium', "N64 DK's Jungle Parkway", 'Wii Mario Circuit', 'N64 Mario Raceway', 'Wii Luigi Circuit', "Wii Toad's Factory", 'Wii Maple Treeway', 'Wii Mushroom Gorge', 'Wii Grumble Volcano', 'SNES Ghost Valley 2', 'SNES Mario Circuit 3', 'Wii Rainbow Road', 'GCN Peach Beach', "N64 Bowser's Castle", 'GBA Shy Guy Beach', 'Wii Koopa Cape', 'DS Desert Hills', 'GBA Bowser Castle 3', 'DS Delfino Square', 'GCN DK Mountain', 'DS Peach Gardens', "Wii Bowser's Castle", 'Wii Moonview Highway', 'Wii DK Summit', 'Wii Dry Dry Ruins', 'N64 Sherbet Land', 'Wii Moo Moo Meadows']


In [16]:
track_diff = [x for x in track_names if x not in tracks_inc]
print(track_diff)

['DS Yoshi Falls', 'GCN Mario Circuit', 'Wii Coconut Mall', 'Wii Daisy Circuit', "Wii Wario's Gold Mine"]


In [17]:
testing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Track   27 non-null     object
 1   AP      27 non-null     object
 2   BT      27 non-null     object
 3   R       27 non-null     object
 4   Engine  27 non-null     object
dtypes: object(5)
memory usage: 1.2+ KB


In [18]:
for track in track_diff:
    testing.loc[len(testing.index)] = [track, '0', '3:00', '0', '150cc']

In [19]:
testing.head()

,Track,AP,BT,R,Engine
0,GCN Waluigi Stadium,2.75,2:11,4,150cc
1,N64 DK's Jungle Parkway,3.40,2:29,5,150cc
2,Wii Mario Circuit,3.00,1:38,1,150cc
3,N64 Mario Raceway,3.00,1:53,1,150cc
4,Wii Luigi Circuit,4.33,1:19,3,150cc


In [21]:
piv = testing.pivot(index = 'Engine', columns='Track')
piv.head()

AP                                                  \
Track  DS Delfino Square DS Desert Hills DS Peach Gardens DS Yoshi Falls   
Engine                                                                     
150cc               6.71            5.00             7.33              0   

                                                              \
Track  GBA Bowser Castle 3 GBA Shy Guy Beach GCN DK Mountain   
Engine                                                         
150cc                 5.60              5.50            6.20   

                                                              ...  \
Track  GCN Mario Circuit GCN Peach Beach GCN Waluigi Stadium  ...   
Engine                                                        ...   
150cc                  0            5.00                2.75  ...   

                    R                                                        \
Track  Wii Koopa Cape Wii Luigi Circuit Wii Maple Treeway Wii Mario Circuit   
Engine                                                                        
150cc               1                 3                 2                 1   

                                                                    \
Track  Wii Moo Moo Meadows Wii Moonview Highway Wii Mushroom Gorge   
Engine                                                               
150cc                    1                    2                  3   

                                                                  
Track  Wii Rainbow Road Wii Toad's Factory Wii Wario's Gold Mine  
Engine                                                            
150cc                 1                  4                     0  

[1 rows x 96 columns]

In [25]:
piv.columns = [' '.join(col).strip() for col in piv.columns.values]
piv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 150cc to 150cc
Data columns (total 96 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AP DS Delfino Square        1 non-null      object
 1   AP DS Desert Hills          1 non-null      object
 2   AP DS Peach Gardens         1 non-null      object
 3   AP DS Yoshi Falls           1 non-null      object
 4   AP GBA Bowser Castle 3      1 non-null      object
 5   AP GBA Shy Guy Beach        1 non-null      object
 6   AP GCN DK Mountain          1 non-null      object
 7   AP GCN Mario Circuit        1 non-null      object
 8   AP GCN Peach Beach          1 non-null      object
 9   AP GCN Waluigi Stadium      1 non-null      object
 10  AP N64 Bowser's Castle      1 non-null      object
 11  AP N64 DK's Jungle Parkway  1 non-null      object
 12  AP N64 Mario Raceway        1 non-null      object
 13  AP N64 Sherbet Land         1 non-null      object


In [37]:
t1 = parseInput(tracks)
print(t1)

[['GCN Waluigi Stadium', '11.25', '2.75', '2:11', '4'], ["N64 DK's Jungle Parkway", '10.20', '3.40', '2:29', '5'], ['Wii Mario Circuit', '10.00', '3.00', '1:38', '1'], ['N64 Mario Raceway', '10.00', '3.00', '1:53', '1'], ['Wii Luigi Circuit', '9.67', '4.33', '1:19', '3'], ["Wii Toad's Factory", '9.50', '3.75', '2:04', '4'], ['Wii Maple Treeway', '9.50', '4.00', '2:37', '2'], ['Wii Mushroom Gorge', '9.33', '4.33', '1:56', '3'], ['Wii Grumble Volcano', '9.17', '4.50', '2:05', '6'], ['SNES Ghost Valley 2', '9.00', '3.67', '1:03', '3'], ['SNES Mario Circuit 3', '8.75', '5.00', '1:32', '4'], ['Wii Rainbow Road', '8.00', '4.00', '2:55', '1'], ['GCN Peach Beach', '8.00', '5.00', '1:28', '2'], ["N64 Bowser's Castle", '7.67', '5.17', '2:59', '6'], ['GBA Shy Guy Beach', '7.50', '5.50', '1:43', '2'], ['Wii Koopa Cape', '7.00', '5.00', '2:47', '1'], ['DS Desert Hills', '7.00', '5.00', '1:58', '1'], ['GBA Bowser Castle 3', '6.40', '5.60', '2:39', '5'], ['DS Delfino Square', '5.86', '6.71', '2:26', 

In [46]:
t2 = formatData(t1, track_names)
t2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 0 to 31
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Track   32 non-null     object
 1   AP      32 non-null     object
 2   BT      32 non-null     object
 3   R       32 non-null     object
 4   Engine  32 non-null     object
dtypes: object(5)
memory usage: 1.5+ KB


In [48]:
t3 = pivotData(t2)
t3.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1 entries, 150cc to 150cc
Data columns (total 96 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   AP DS Delfino Square        1 non-null      object
 1   AP DS Desert Hills          1 non-null      object
 2   AP DS Peach Gardens         1 non-null      object
 3   AP DS Yoshi Falls           1 non-null      object
 4   AP GBA Bowser Castle 3      1 non-null      object
 5   AP GBA Shy Guy Beach        1 non-null      object
 6   AP GCN DK Mountain          1 non-null      object
 7   AP GCN Mario Circuit        1 non-null      object
 8   AP GCN Peach Beach          1 non-null      object
 9   AP GCN Waluigi Stadium      1 non-null      object
 10  AP N64 Bowser's Castle      1 non-null      object
 11  AP N64 DK's Jungle Parkway  1 non-null      object
 12  AP N64 Mario Raceway        1 non-null      object
 13  AP N64 Sherbet Land         1 non-null      object


In [49]:
t3.head()

,AP DS Delfino Square,AP DS Desert Hills,AP DS Peach Gardens,AP DS Yoshi Falls,AP GBA Bowser Castle 3,AP GBA Shy Guy Beach,AP GCN DK Mountain,AP GCN Mario Circuit,AP GCN Peach Beach,AP GCN Waluigi Stadium,...,R Wii Koopa Cape,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine
Engine,,,,,,,,,,,,,,,,,,,,,
150cc,6.71,5.00,7.33,0,5.60,5.50,6.20,0,5.00,2.75,...,1,3,2,1,1,2,3,1,4,0


In [51]:
t3.describe()

,AP DS Delfino Square,AP DS Desert Hills,AP DS Peach Gardens,AP DS Yoshi Falls,AP GBA Bowser Castle 3,AP GBA Shy Guy Beach,AP GCN DK Mountain,AP GCN Mario Circuit,AP GCN Peach Beach,AP GCN Waluigi Stadium,...,R Wii Koopa Cape,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine
count,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
unique,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
top,6.71,5.00,7.33,0,5.60,5.50,6.20,0,5.00,2.75,...,1,3,2,1,1,2,3,1,4,0
freq,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [59]:
t4 = typeInput(t3)
t4.info()

AttributeError: 'float' object has no attribute 'split'

In [60]:
t4.head()

,AP DS Delfino Square,AP DS Desert Hills,AP DS Peach Gardens,AP DS Yoshi Falls,AP GBA Bowser Castle 3,AP GBA Shy Guy Beach,AP GCN DK Mountain,AP GCN Mario Circuit,AP GCN Peach Beach,AP GCN Waluigi Stadium,...,R Wii Koopa Cape,R Wii Luigi Circuit,R Wii Maple Treeway,R Wii Mario Circuit,R Wii Moo Moo Meadows,R Wii Moonview Highway,R Wii Mushroom Gorge,R Wii Rainbow Road,R Wii Toad's Factory,R Wii Wario's Gold Mine
Engine,,,,,,,,,,,,,,,,,,,,,
150cc,671,500,733,0,560,550,620,0,500,275,...,1,3,2,1,1,2,3,1,4,0
